<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/openai_mistral_test_disitll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flash-attn --no-build-isolation -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load base + adapter (uses FP16 for efficiency)
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    dtype=torch.float16,
    device_map="auto",
    attn_implementation="flash_attention_2"  # Optional: For speed
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(base_model, "frankmorales2020/mistral-7b-gpt-oss-20b-distilled")

# Test prompt (from your dataset)
prompt = "### Instruction:\nWrite a Python function to compute Fibonacci numbers efficiently.\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

In [2]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Write a Python function to compute Fibonacci numbers efficiently.

### Response:
```
#!/usr/bin/python

def fib(n):
    if n <= 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

# Driver code
n = int(input("Enter the number of terms: "))
print("Fibonacci series:")
for i in range(n):
    print(fib(i))
```

### Explanation:
- The function `fib` takes an integer `n` as input and returns the `n`th Fibonacci number.
- If `n` is less than or equal to 1, the function returns `n` directly.
- Otherwise, the function recursively calls itself to compute the `n-1`th and `n-2`th Fibonacci numbers and adds them to obtain the `n`th Fibonacci number.
- The driver code prompts the user to enter the number of terms and then prints the Fibonacci series up to that number.
- The output of the program will be a list of Fibonacci numbers up to the specified number of terms.

### Complexity Analysis:
- Time Complexity: The time complexity of the function `fib` is exponential, 